In [1]:
import pandas as pd
from sportsreference.ncaab.teams import Teams
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import numpy as np 
import matplotlib.pyplot as plt
import urllib.request


from datetime import datetime
import requests
import os
from bs4 import BeautifulSoup

In [157]:
df = pd.read_csv('NBA_2016_thru_2022_seasons.csv')
df = df[~df['Unnamed: 4'].isna()].reset_index(drop=True)
df['Date'] = pd.to_datetime(df['Date'])
df.sort_values(by=['Date'], ascending=True, inplace=True)
df.drop(columns = ['Unnamed: 0', 'Unnamed: 4'], inplace=True)
df.rename(columns={'Unnamed: 5':'home_away',
                  'Unnamed: 7': 'result',
                  'Unnamed: 8':'ot'}, inplace=True)



# df = df[df['home_away'].isna()].reset_index(drop=True)


# df.drop(columns=['home_away'], inplace=True)

# df['game_key'] = df['home_team'].astype(str) + '_' + df['G'].astype(str) + '_' + df['season'].astype(str)


team_map = pd.read_csv('team_name_mapping.csv')

df = df.merge(team_map, left_on='Opponent', right_on='full_name', how='inner')
df.drop(columns=['Opponent', 'full_name'], inplace=True)

df.rename(columns= {'abbr':'opponent'}, inplace=True)

# df.drop(columns=['home_awa7y],inplace=True)

# df.rename(columns = {'home_team':'home_abbr'}, inplace=True)

df['game_key'] = ''

for i in range(len(df)):
    if (df.loc[i, 'home_away'] == '@'):
        df.loc[i, 'game_key'] = df.loc[i, 'teamgame'] + '_at_' + df.loc[i, 'opponent'] + '_' + datetime.strftime(df.loc[i, 'Date'], '%d_%m_%Y')
    else:
        df.loc[i, 'game_key'] = df.loc[i, 'opponent'] + '_at_' + df.loc[i, 'teamgame'] + '_' + datetime.strftime(df.loc[i, 'Date'], '%d_%m_%Y')
        
df['ppg'] = 0

In [191]:
dff = pd.DataFrame()


for team in df['teamgame'].unique():
    dft = pd.DataFrame(df[df['teamgame'] == team].reset_index(drop=True))
    
    for szn in dft['season'].unique():
        dfts = pd.DataFrame(dft[dft['season'] == szn])
        
        dfts.sort_values(by=['Date'], inplace=True)
        
        dfts.reset_index(drop=True, inplace=True)
        
        for i in range(len(dfts)-1):
            if (dfts.loc[i, 'G'] > 1):
                dfts.loc[i, 'ppg'] = np.mean(dfts.loc[0:i,'Tm'])
                
        dff = pd.concat([dff,dfts])

dff.reset_index(drop=True, inplace=True)

df = dff.copy()

In [193]:
away = df[df['home_away'] == '@'].reset_index(drop=True)
home = df[df['home_away'] != '@'].reset_index(drop=True)

away['wl_total'] = away['G']-1
home['wl_total'] = home['G']-1

away['win_pct'] = 0.0
home['win_pct'] = 0.0

for i in range(len(away)):
    if (away.loc[i,'result'] == 'L'):
        if (away.loc[i, 'wl_total'] == 0):
            away.loc[i, 'win_pct'] = 0
        elif (away.loc[i, 'wl_total'] > 0):
            away.loc[i, 'win_pct'] = away.loc[i, 'W']/(away.loc[i, 'G']-1)
    elif (away.loc[i,'result'] == 'W'):
        if (away.loc[i, 'wl_total'] == 0):
            away.loc[i, 'win_pct'] = 0
        else:
            away.loc[i, 'win_pct'] = (away.loc[i, 'W']-1)/(away.loc[i, 'G']-1)

away.rename(columns={'ppg':'away_ppg'}, inplace=True)
  
for i in range(len(home)):
    if (home.loc[i,'result'] == 'L'):
        if (home.loc[i, 'wl_total'] == 0):
            home.loc[i, 'win_pct'] = 0
        elif (home.loc[i, 'wl_total'] > 0):
            home.loc[i, 'win_pct'] = home.loc[i, 'W']/(home.loc[i, 'G']-1)
    elif (home.loc[i,'result'] == 'W'):
        if (home.loc[i, 'wl_total'] == 0):
            home.loc[i, 'win_pct'] = 0
        else:
            home.loc[i, 'win_pct'] = (home.loc[i, 'W']-1)/(home.loc[i, 'G']-1) 

            
home.rename(columns={'ppg':'home_ppg'}, inplace=True)

In [194]:
home

,G,Date,Start (ET),home_away,result,ot,Tm,Opp,W,L,Streak,teamgame,season,opponent,game_key,home_ppg,wl_total,win_pct
0,1,2015-10-27,8:00p,NaN,L,NaN,94.0,106.0,0.0,1.0,L 1,ATL,2016,DET,DET_at_ATL_27_10_2015,0.000000,0,0.000000
1,3,2015-10-30,8:00p,NaN,W,NaN,97.0,94.0,2.0,1.0,W 2,ATL,2016,CHO,CHO_at_ATL_30_10_2015,101.000000,2,0.500000
2,6,2015-11-04,8:00p,NaN,W,NaN,101.0,87.0,5.0,1.0,W 5,ATL,2016,BRK,BRK_at_ATL_04_11_2015,99.333333,5,0.800000
3,8,2015-11-07,7:30p,NaN,W,NaN,114.0,99.0,7.0,1.0,W 7,ATL,2016,WAS,WAS_at_ATL_07_11_2015,103.875000,7,0.857143
4,9,2015-11-09,8:00p,NaN,L,NaN,107.0,117.0,7.0,2.0,L 1,ATL,2016,MIN,MIN_at_ATL_09_11_2015,104.222222,8,0.875000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7750,34,2022-01-01,7:00p,NaN,W,OT,117.0,116.0,6.0,28.0,W 1,DET,2022,SAS,SAS_at_DET_01_01_2022,100.735294,33,0.151515
7751,38,2022-01-08,7:00p,NaN,W,NaN,97.0,92.0,8.0,30.0,W 1,DET,2022,ORL,ORL_at_DET_08_01_2022,100.947368,37,0.189189
7752,39,2022-01-10,7:00p,NaN,W,NaN,126.0,116.0,9.0,30.0,W 2,DET,2022,UTA,UTA_at_DET_10_01_2022,101.589744,38,0.210526
7753,41,2022-01-14,7:00p,NaN,W,NaN,103.0,87.0,10.0,31.0,W 1,DET,2022,TOR,TOR_at_DET_14_01_2022,101.268293,40,0.225000


In [195]:
df[(df['season'] == 2016) & (df['teamgame'] == 'ATL')]

,G,Date,Start (ET),home_away,result,ot,Tm,Opp,W,L,Streak,teamgame,season,opponent,game_key,ppg
0,1,2015-10-27,8:00p,NaN,L,NaN,94.0,106.0,0.0,1.0,L 1,ATL,2016,DET,DET_at_ATL_27_10_2015,0.000000
1,2,2015-10-29,8:00p,@,W,NaN,112.0,101.0,1.0,1.0,W 1,ATL,2016,NYK,ATL_at_NYK_29_10_2015,103.000000
2,3,2015-10-30,8:00p,NaN,W,NaN,97.0,94.0,2.0,1.0,W 2,ATL,2016,CHO,CHO_at_ATL_30_10_2015,101.000000
3,4,2015-11-01,2:00p,@,W,NaN,94.0,92.0,3.0,1.0,W 3,ATL,2016,CHO,ATL_at_CHO_01_11_2015,99.250000
4,5,2015-11-03,7:30p,@,W,NaN,98.0,92.0,4.0,1.0,W 4,ATL,2016,MIA,ATL_at_MIA_03_11_2015,99.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,78,2016-04-05,8:00p,NaN,W,NaN,103.0,90.0,46.0,32.0,W 1,ATL,2016,PHO,PHO_at_ATL_05_04_2016,102.923077
78,79,2016-04-07,8:00p,NaN,W,NaN,95.0,87.0,47.0,32.0,W 2,ATL,2016,TOR,TOR_at_ATL_07_04_2016,102.822785
79,80,2016-04-09,7:30p,NaN,W,NaN,118.0,107.0,48.0,32.0,W 3,ATL,2016,BOS,BOS_at_ATL_09_04_2016,103.012500
80,81,2016-04-11,7:00p,@,L,NaN,94.0,109.0,48.0,33.0,L 1,ATL,2016,CLE,ATL_at_CLE_11_04_2016,102.901235


In [196]:
away_pcts = pd.DataFrame(away[['game_key', 'win_pct', 'away_ppg']])

away_pcts.rename(columns = 
                {
                    'win_pct':'away_win_pct'
                },
                 
                inplace=True)

home.rename(columns = {'win_pct':'home_win_pct'}, inplace=True)

In [197]:
allg = home.merge(away_pcts, on='game_key', how='inner')

allg['pt_diff'] = allg['Tm'] - allg['Opp']

In [198]:
allg

,G,Date,Start (ET),home_away,result,ot,Tm,Opp,W,L,...,teamgame,season,opponent,game_key,home_ppg,wl_total,home_win_pct,away_win_pct,away_ppg,pt_diff
0,1,2015-10-27,8:00p,NaN,L,NaN,94.0,106.0,0.0,1.0,...,ATL,2016,DET,DET_at_ATL_27_10_2015,0.000000,0,0.000000,0.000000,0.000000,-12.0
1,3,2015-10-30,8:00p,NaN,W,NaN,97.0,94.0,2.0,1.0,...,ATL,2016,CHO,CHO_at_ATL_30_10_2015,101.000000,2,0.500000,0.000000,94.000000,3.0
2,6,2015-11-04,8:00p,NaN,W,NaN,101.0,87.0,5.0,1.0,...,ATL,2016,BRK,BRK_at_ATL_04_11_2015,99.333333,5,0.800000,0.000000,89.800000,14.0
3,8,2015-11-07,7:30p,NaN,W,NaN,114.0,99.0,7.0,1.0,...,ATL,2016,WAS,WAS_at_ATL_07_11_2015,103.875000,7,0.857143,0.600000,102.500000,15.0
4,9,2015-11-09,8:00p,NaN,L,NaN,107.0,117.0,7.0,2.0,...,ATL,2016,MIN,MIN_at_ATL_09_11_2015,104.222222,8,0.875000,0.600000,101.833333,-10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7750,34,2022-01-01,7:00p,NaN,W,OT,117.0,116.0,6.0,28.0,...,DET,2022,SAS,SAS_at_DET_01_01_2022,100.735294,33,0.151515,0.411765,111.485714,1.0
7751,38,2022-01-08,7:00p,NaN,W,NaN,97.0,92.0,8.0,30.0,...,DET,2022,ORL,ORL_at_DET_08_01_2022,100.947368,37,0.189189,0.179487,101.700000,5.0
7752,39,2022-01-10,7:00p,NaN,W,NaN,126.0,116.0,9.0,30.0,...,DET,2022,UTA,UTA_at_DET_10_01_2022,101.589744,38,0.210526,0.700000,115.707317,10.0
7753,41,2022-01-14,7:00p,NaN,W,NaN,103.0,87.0,10.0,31.0,...,DET,2022,TOR,TOR_at_DET_14_01_2022,101.268293,40,0.225000,0.526316,107.282051,16.0


In [84]:
df[(df['Opp']==106.0) & (df['Tm']==94)]

,G,Date,Start (ET),home_away,away_team,result,ot,Tm,Opp,W,L,Streak,home_abbr,season,away_abbr,game_key
0,1,2015-10-27,8:00p,NaN,Detroit Pistons,L,NaN,94.0,106.0,0.0,1.0,L 1,ATL,2016,DET,DET_at_ATL_2015-10-27
1437,11,2015-11-15,5:00p,@,Charlotte Hornets,L,NaN,94.0,106.0,4.0,7.0,L 5,POR,2016,CHO,CHO_at_POR_2015-11-15
14536,12,2015-11-20,10:30p,@,Golden State Warriors,L,NaN,94.0,106.0,8.0,4.0,L 1,CHI,2016,GSW,GSW_at_CHI_2015-11-20
9049,44,2016-01-20,8:30p,@,Dallas Mavericks,L,OT,94.0,106.0,13.0,31.0,L 2,MIN,2016,DAL,DAL_at_MIN_2016-01-20
3496,50,2016-02-05,7:00p,@,Washington Wizards,L,NaN,94.0,106.0,7.0,43.0,L 4,PHI,2016,WAS,WAS_at_PHI_2016-02-05
4952,61,2016-03-05,7:00p,NaN,Utah Jazz,L,NaN,94.0,106.0,23.0,38.0,L 4,NOP,2016,UTA,UTA_at_NOP_2016-03-05
610,71,2016-03-24,7:30p,@,New York Knicks,L,NaN,94.0,106.0,36.0,35.0,L 2,CHI,2016,NYK,NYK_at_CHI_2016-03-24
7497,32,2016-12-27,7:30p,NaN,Oklahoma City Thunder,L,NaN,94.0,106.0,10.0,22.0,L 2,MIA,2017,OKC,OKC_at_MIA_2016-12-27
11994,34,2017-01-04,8:00p,NaN,Chicago Bulls,L,NaN,94.0,106.0,26.0,8.0,L 1,CLE,2017,CHI,CHI_at_CLE_2017-01-04
14709,73,2017-03-26,8:00p,@,Golden State Warriors,L,NaN,94.0,106.0,40.0,33.0,L 3,MEM,2017,GSW,GSW_at_MEM_2017-03-26


In [57]:
away = df[df['home_away'] == '@'].reset_index(drop=True)
home = df[df['home_away'] != '@'].reset_index(drop=True)



In [50]:
sorted(df['home_team'].unique())

['ATL',
 'BOS',
 'BRK',
 'CHI',
 'CHO',
 'CLE',
 'DAL',
 'DEN',
 'DET',
 'GSW',
 'HOU',
 'IND',
 'LAC',
 'LAL',
 'MEM',
 'MIA',
 'MIL',
 'MIN',
 'NOP',
 'NYK',
 'OKC',
 'ORL',
 'PHI',
 'PHO',
 'POR',
 'SAC',
 'SAS',
 'TOR',
 'UTA',
 'WAS']

In [29]:
df['prior_wl_total'] = 0

df['home_tm_win_pct'] = 0.0


In [ ]:
for team in df['home_team']

In [ ]:

# for i in range(len(df)):
#     if (df.loc[i,'result'] == 'W'):
#         df.loc[i, 'prior_wl_total'] = (df.loc[i, 'W']-1)+df.loc[i, 'L']
#     else:
#         df.loc[i, 'prior_wl_total'] = (df.loc[i, 'L']-1)+df.loc[i, 'W']
    
#     if df.loc[i, 'prior_wl_total'] != 0:
#         df['home_tm_win_pct']

In [ ]:
len(df[df['']])